# Rollout

- Ref: https://learnk8s.io/kubernetes-rollbacks
- Ref: https://ithelp.ithome.com.tw/articles/10195639

滾動升級/回滾可以讓你在不終止現有服務情況下，進行升級。



In [ ]:
# 確保你的minikube有啟動
! [[ -z $(kubectl get nodes) ]] && minikube start 
# 列印目前k8s節點
! kubectl get nodes                    

😄  minikube v1.28.0 on Ubuntu 22.04 (vbox/amd64)
✨  Using the docker driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🏃  Updating the running docker "minikube" container ...
🐳  Preparing Kubernetes v1.25.3 on Docker 20.10.20 ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


In [12]:
# get current contexts
! kubectl config get-contexts

CURRENT   NAME       CLUSTER    AUTHINFO   NAMESPACE
*         minikube   minikube   minikube   default


In [ ]:
# delete all stuff
! kubectl delete service --all
! kubectl delete pods --all
! kubectl delete replica --all
! kubectl delete deployment --all

service "kubernetes" deleted
pod "hostname-5477bcd76c-gbchf" deleted
pod "hostname-5477bcd76c-l8lmk" deleted
pod "hostname-5477bcd76c-th95n" deleted
pod "hostname-5477bcd76c-wq4rl" deleted
pod "hostname-5477bcd76c-z6bdp" deleted
pod "hostname-5477bcd76c-zhd6n" deleted
^C
error: the server doesn't have a resource type "replica"
No resources found


the server doesn't have a resource type "rollout"


In [61]:
# 建立一個 hostname deployment
# replicas 為 3
# image 為 hostname:v1
! kubectl delete deployment --all
! kubectl create deployment hostname --replicas=3 --image=brokenpen/hostname:v1 

deployment.apps "hostname" deleted
deployment.apps/hostname created


In [71]:
! kubectl rollout --help

Manage the rollout of one or many resources.
  
 Valid resource types include:

  *  deployments
  *  daemonsets
  *  statefulsets

Examples:
  # Rollback to the previous deployment
  kubectl rollout undo deployment/abc
  
  # Check the rollout status of a daemonset
  kubectl rollout status daemonset/foo
  
  # Restart a deployment
  kubectl rollout restart deployment/abc
  
  # Restart deployments with the app=nginx label
  kubectl rollout restart deployment --selector=app=nginx

Available Commands:
  history       View rollout history
  pause         Mark the provided resource as paused
  restart       Restart a resource
  resume        Resume a paused resource
  status        Show the status of the rollout
  undo          Undo a previous rollout

Usage:
  kubectl rollout SUBCOMMAND [options]

Use "kubectl <command> --help" for more information about a given command.
Use "kubectl options" for a list of global command-line options (applies to all
commands).


In [62]:
! kubectl rollout history deployment hostname

deployment.apps/hostname 
REVISION  CHANGE-CAUSE
1         <none>



In [63]:
! kubectl rollout history deployment hostname --revision=1

deployment.apps/hostname with revision #1
Pod Template:
  Labels:	app=hostname
	pod-template-hash=7c99c74d96
  Containers:
   hostname:
    Image:	brokenpen/hostname:v1
    Port:	<none>
    Host Port:	<none>
    Environment:	<none>
    Mounts:	<none>
  Volumes:	<none>



In [66]:
! kubectl annotate deployment hostname kubernetes.io/change-cause="created with v1 image" 

deployment.apps/hostname annotated


In [82]:
! kubectl rollout history deployment hostname

deployment.apps/hostname 
REVISION  CHANGE-CAUSE
1         created with v1 image



In [16]:
# 列印這個 deployment 資訊
! kubectl get deployment hostname --show-labels  -o wide

NAME       READY   UP-TO-DATE   AVAILABLE   AGE   CONTAINERS   IMAGES                  SELECTOR       LABELS
hostname   0/3     3            0           0s    hostname     brokenpen/hostname:v1   app=hostname   app=hostname


In [17]:
# 列印目前有的 replicaset
# 也就是這個deployment 所建立的 replicaset
! kubectl get replicaset -o wide

NAME                  DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES                  SELECTOR
hostname-7c99c74d96   3         3         0       1s    hostname     brokenpen/hostname:v1   app=hostname,pod-template-hash=7c99c74d96


In [18]:
! kubectl get pod -o wide

NAME                        READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
hostname-7c99c74d96-4m7xb   1/1     Running   0          7s    172.17.0.4   minikube   <none>           <none>
hostname-7c99c74d96-ps6nf   1/1     Running   0          7s    172.17.0.5   minikube   <none>           <none>
hostname-7c99c74d96-q2dlx   1/1     Running   0          7s    172.17.0.3   minikube   <none>           <none>


<img class="img" style="width:480px;" src="https://learnk8s.io/a/e2d1c566c4b5cd59d9fa8e0de547344b.svg"></img>

In [80]:
# 把deployment的imgage
# 改成 hostname:v2
! kubectl set image deployment/hostname \
    hostname=brokenpen/hostname:v2 \

error: all resources must be specified before image changes: \


In [54]:
! kubectl annotate deployment hostname kubernetes.io/change-cause="change image to v2" 

deployment.apps/hostname annotated


In [81]:
! kubectl rollout history deployment hostname 

deployment.apps/hostname 
REVISION  CHANGE-CAUSE
1         created with v1 image



In [75]:
! kubectl rollout history deployment hostname --revision=1

deployment.apps/hostname with revision #1
Pod Template:
  Labels:	app=hostname
	pod-template-hash=7c99c74d96
  Annotations:	kubernetes.io/change-cause: created with v1 image
  Containers:
   hostname:
    Image:	brokenpen/hostname:v1
    Port:	<none>
    Host Port:	<none>
    Environment:	<none>
    Mounts:	<none>
  Volumes:	<none>



In [22]:
! kubectl get replicaset -o wide 
! sleep 2; echo; echo;
! kubectl get rs -o wide
! sleep 2; echo; echo;
! kubectl get rs -o wide
! sleep 2; echo; echo;
! kubectl get rs -o wide
! sleep 2; echo; echo;

NAME                  DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES                  SELECTOR
hostname-5477bcd76c   3         3         3       31s   hostname     brokenpen/hostname:v2   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c74d96   0         0         0       49s   hostname     brokenpen/hostname:v1   app=hostname,pod-template-hash=7c99c74d96


NAME                  DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES                  SELECTOR
hostname-5477bcd76c   3         3         3       34s   hostname     brokenpen/hostname:v2   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c74d96   0         0         0       52s   hostname     brokenpen/hostname:v1   app=hostname,pod-template-hash=7c99c74d96


NAME                  DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES                  SELECTOR
hostname-5477bcd76c   3         3         3       36s   hostname     brokenpen/hostname:v2   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c7


<img class="img" style="width:480px;" src="img/rollout-ready-2.png"></img>

---

<img class="img" style="width:480px;" src="img/rollout-ready-3.png"></img>

---

<img class="img" style="width:480px;" src="img/rollout-ready-4.png"></img>

---

<img class="img" style="width:480px;" src="img/rollout-ready-5.png"></img>


In [23]:
! kubectl rollout status deployment/hostname

deployment "hostname" successfully rolled out


In [24]:
! kubectl get deployments hostname -o wide

NAME       READY   UP-TO-DATE   AVAILABLE   AGE    CONTAINERS   IMAGES                  SELECTOR
hostname   3/3     3            3           100s   hostname     brokenpen/hostname:v2   app=hostname


In [26]:
# image v2 的 replciaSet 已經建立
# 而舊的 image v1 replicaSet 會保留著
! kubectl get rs -o wide

NAME                  DESIRED   CURRENT   READY   AGE     CONTAINERS   IMAGES                  SELECTOR
hostname-5477bcd76c   3         3         3       119s    hostname     brokenpen/hostname:v2   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c74d96   0         0         0       2m17s   hostname     brokenpen/hostname:v1   app=hostname,pod-template-hash=7c99c74d96


In [27]:
! kubectl get pods 

NAME                        READY   STATUS    RESTARTS   AGE
hostname-5477bcd76c-4k28c   1/1     Running   0          116s
hostname-5477bcd76c-5f2lf   1/1     Running   0          114s
hostname-5477bcd76c-xt7nw   1/1     Running   0          2m2s


In [37]:
! kubectl rollout history deployment hostname

deployment.apps/hostname 
REVISION  CHANGE-CAUSE
3         <none>
4         kubectl set image deployment/hostname hostname=brokenpen/hostname:v2 --record=true



In [33]:
! kubectl rollout undo deployment/hostname 

error: unknown flag: --record
See 'kubectl rollout undo --help' for usage.


In [34]:
! kubectl get rs -o wide

NAME                  DESIRED   CURRENT   READY   AGE     CONTAINERS   IMAGES                  SELECTOR
hostname-5477bcd76c   0         0         0       3m28s   hostname     brokenpen/hostname:v2   app=hostname,pod-template-hash=5477bcd76c
hostname-7c99c74d96   3         3         3       3m46s   hostname     brokenpen/hostname:v1   app=hostname,pod-template-hash=7c99c74d96


In [35]:
! kubectl get pods 

NAME                        READY   STATUS        RESTARTS   AGE
hostname-5477bcd76c-4k28c   1/1     Terminating   0          3m28s
hostname-5477bcd76c-5f2lf   1/1     Terminating   0          3m26s
hostname-5477bcd76c-xt7nw   1/1     Terminating   0          3m34s
hostname-7c99c74d96-jmpp8   1/1     Running       0          11s
hostname-7c99c74d96-n48wr   1/1     Running       0          9s
hostname-7c99c74d96-z82rj   1/1     Running       0          13s


In [ ]:
! kubectl rollout pause deployment hostname

In [ ]:
! kubectl rollout resume deployment hostname